In [ ]:
!pip install tweepy praw google-api-python-client transformers torch pandas numpy matplotlib folium

In [ ]:
!pip install praw transformers torch pandas numpy matplotlib folium

In [ ]:
reddit_client_id = "cFfHjWEEzqDVNiSkXicfwA"
reddit_client_secret = "YpJR2-0eeSoqc3A3d1EWwhYdM-O2-w"
reddit_user_agent = "CloudSEK_Hackathon by /u/Jazzlike-Age6536"

In [ ]:
import praw
import pandas as pd

# Authenticate with Reddit
reddit = praw.Reddit(
    client_id=reddit_client_id,
    client_secret=reddit_client_secret,
    user_agent=reddit_user_agent
)

# Verify authentication
try:
    print("Authenticated as:", reddit.user.me())
except Exception as e:
    print("Authentication check failed:", e)

# Search subreddits with error handling
subreddits = "worldnews+geopolitics+india"
reddit_posts = []
try:
    for submission in reddit.subreddit(subreddits).search("India Pakistan terrorism OR Kashmir conflict OR border attack", limit=50):
        reddit_posts.append({
            "platform": "Reddit",
            "text": submission.title + " " + submission.selftext,
            "user": submission.author.name if submission.author else "Anonymous",
            "location": "N/A",
            "created_at": submission.created_utc
        })
    print(f"Successfully collected {len(reddit_posts)} posts.")
except Exception as e:
    print("Error during scraping:", e)
    raise

# Save to DataFrame
reddit_df = pd.DataFrame(reddit_posts)
reddit_df.to_csv("reddit_osint_data.csv", index=False)
print("Data saved to reddit_osint_data.csv")
print(reddit_df.head())

ModuleNotFoundError: No module named 'praw'

In [ ]:
!pip install timeout-decorator

  Preparing metadata (setup.py) ... done
  Created wheel for timeout-decorator: filename=timeout_decorator-0.5.0-py3-none-any.whl size=5006 sha256=64b7fba237c620828ce1d3c2f95cc837c65041984555075ea3e4778e5afd5197
  Stored in directory: /root/.cache/pip/wheels/aa/cd/d1/51736c6b95846b2613a520ce146a8f305c4016a987bc9faec7
Successfully built timeout-decorator


In [ ]:
from transformers import pipeline, AutoTokenizer
import pandas as pd
import re

# Load data
data = pd.read_csv("reddit_osint_data.csv")
data = data.head(50)

# Step 4.1: Threat Detection using Toxic Detection + Keywords
print("Running threat detection...")
try:
    tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
    classifier = pipeline("text-classification", model="unitary/toxic-bert")
    print("Using unitary/toxic-bert model")
except Exception as e:
    print(f"Failed to load toxic model: {e}. Falling back to cardiffnlp...")
    tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
    classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment-latest")

max_length = 512

# Function to truncate text
def truncate_text(text, max_len=max_length):
    encoded = tokenizer.encode(text, truncation=True, max_length=max_len, return_tensors="pt")
    decoded = tokenizer.decode(encoded[0], skip_special_tokens=True)
    return decoded

# Define refined threat-related keywords
threat_keywords = [
    "terrorist attack", "bomb explosion", "militant strike", "violence outbreak",
    "kill operation", "assault plan", "terror plot","terrorist", "armed conflict"
]

# Check for threat keywords in text
def has_threat_keywords(text):
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in threat_keywords)

# Classify posts with toxic model
def classify_text(text):
    try:
        truncated_text = truncate_text(text)
        result = classifier(truncated_text)[0]
        if "toxic-bert" in classifier.model.name_or_path:
            # Map toxic classification to threat score
            return result["score"] if result["label"] == "toxic" else 0
        else:  # Fallback to cardiffnlp
            result = classifier(truncated_text)[0]
            if result["label"] == "negative":
                return result["score"]
            elif result["label"] == "neutral":
                return result["score"] * 0.5
            else:
                return 0
    except Exception as e:
        print(f"Error classifying text: {text[:50]}... | Error: {e}")
        return 0

data["threat_score"] = data["text"].apply(classify_text)
# Flag threats based on BOTH score > 0.5 AND keywords
data["is_threat"] = (data["threat_score"] > 0.5) & data["text"].apply(has_threat_keywords)

# Print all posts with threat scores
print("All Posts with Threat Scores:")
print(data[["platform", "text", "threat_score", "is_threat"]])

# Step 4.2: Entity Extraction (Locations)
print("\nExtracting locations...")
ner = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)
ner_tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

def truncate_text_for_ner(text, max_len=max_length):
    encoded = ner_tokenizer.encode(text, truncation=True, max_length=max_len, return_tensors="pt")
    decoded = ner_tokenizer.decode(encoded[0], skip_special_tokens=True)
    return decoded

def extract_entities(text):
    try:
        truncated_text = truncate_text_for_ner(text)
        entities = ner(truncated_text)
        locations = [e["word"] for e in entities if e["entity_group"] == "LOC"]
        cleaned_locations = [re.sub(r'##', '', loc) for loc in locations]
        return ", ".join(cleaned_locations) if cleaned_locations else "N/A"
    except Exception as e:
        print(f"Error extracting entities from text: {text[:50]}... | Error: {e}")
        return "N/A"

data["locations"] = data["text"].apply(extract_entities)
print("Posts with Extracted Locations:")
print(data[["platform", "text", "locations"]])

# Save the analyzed data
data.to_csv("reddit_osint_analyzed.csv", index=False)
print("Analyzed data saved to reddit_osint_analyzed.csv")

Running threat detection...


Device set to use cpu


Using unitary/toxic-bert model


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


All Posts with Threat Scores:
   platform                                               text  threat_score  \
0    Reddit  [R] Megathread III: India-Pakistan Border Skir...      0.000766   
1    Reddit  Are there growing signs of instability in Paki...      0.002829   
2    Reddit  A rundown of the ideological roots and history...      0.000699   
3    Reddit  International Diplomatic Response to Indian Ai...      0.000712   
4    Reddit  Neoliberalism, Crony Capitalists, And The Indi...      0.000819   
5    Reddit  What are some of the mind blowing operations o...      0.002264   
6    Reddit  Found something that you should know about Roh...      0.001188   
7    Reddit  India-Pakistan tensions rise as India announce...      0.000875   
8    Reddit  Pahalgam terror attack: UN urges India and Pak...      0.002354   
9    Reddit  India-Pakistan Tensions On Verge Of Erupting I...      0.028776   
10   Reddit  Kashmiri students attacked in Punjab as India ...      0.005165   
11   Reddi

Device set to use cpu
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/token_classification.py:170: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.SIMPLE"` instead.
  warnings.warn(


Posts with Extracted Locations:
   platform                                               text  \
0    Reddit  [R] Megathread III: India-Pakistan Border Skir...   
1    Reddit  Are there growing signs of instability in Paki...   
2    Reddit  A rundown of the ideological roots and history...   
3    Reddit  International Diplomatic Response to Indian Ai...   
4    Reddit  Neoliberalism, Crony Capitalists, And The Indi...   
5    Reddit  What are some of the mind blowing operations o...   
6    Reddit  Found something that you should know about Roh...   
7    Reddit  India-Pakistan tensions rise as India announce...   
8    Reddit  Pahalgam terror attack: UN urges India and Pak...   
9    Reddit  India-Pakistan Tensions On Verge Of Erupting I...   
10   Reddit  Kashmiri students attacked in Punjab as India ...   
11   Reddit  In a secret deal, Pakistan allowed American dr...   
12   Reddit  A group of Indians were filmed chanting ‘death...   
13   Reddit  Key ISI Figure Behind 2018 J&K 

In [ ]:
from transformers import pipeline, AutoTokenizer
import pandas as pd
import re

# Load data (limited to 20 posts)
data = pd.read_csv("reddit_osint_data.csv")
data = data.head(20)

# Step 4.1: Threat Detection using Toxic Detection + Keywords
print("Running threat detection...")
try:
    tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
    classifier = pipeline("text-classification", model="unitary/toxic-bert")
    print("Using unitary/toxic-bert model")
except Exception as e:
    print(f"Failed to load toxic model: {e}. Falling back to nlptown sentiment...")
    tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
    classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

max_length = 512

# Function to truncate text
def truncate_text(text, max_len=max_length):
    encoded = tokenizer.encode(text, truncation=True, max_length=max_len, return_tensors="pt")
    decoded = tokenizer.decode(encoded[0], skip_special_tokens=True)
    return decoded

# Define threat-related keywords
threat_keywords = [
    "terrorist", "attack", "bomb", "explosion", "militant", "strike", "violence",
    "kill", "assault", "terror", "conflict", "death", "threat", "war"
]

# Check for any threat keyword in text
def has_threat_keywords(text):
    text_lower = text.lower()
    return any(keyword in text_lower for keyword in threat_keywords)

# Classify posts with toxic model
def classify_text(text):
    try:
        truncated_text = truncate_text(text)
        result = classifier(truncated_text)[0]
        if "toxic-bert" in classifier.model.name_or_path:
            score = result["score"] if result["label"] == "toxic" else 0
        else:  # Fallback to nlptown sentiment
            # This model outputs star ratings (1 to 5 stars); map to threat score
            label = int(result["label"].split()[0])  # e.g., "1 star" → 1
            score = (5 - label) / 4  # Map 1 star to 1 (high threat), 5 stars to 0 (low threat)

        # If keywords are present, ensure a minimum score to avoid missing threats
        if has_threat_keywords(text):
            score = max(score, 0.1)  # Minimum score if keywords are present
        return score
    except Exception as e:
        print(f"Error classifying text: {text[:50]}... | Error: {e}")
        return 0

data["threat_score"] = data["text"].apply(classify_text)
# Flag threats based on score > 0.05 AND keywords
data["is_threat"] = (data["threat_score"] > 0.05) & data["text"].apply(has_threat_keywords)

# Print all posts with threat scores
print("All Posts with Threat Scores:")
print(data[["platform", "text", "threat_score", "is_threat"]])

# Step 4.2: Entity Extraction (Locations)
print("\nExtracting locations...")
ner = pipeline("ner", model="dslim/bert-base-NER", grouped_entities=True)
ner_tokenizer = AutoTokenizer.from_pretrained("dslim/bert-base-NER")

def truncate_text_for_ner(text, max_len=max_length):
    encoded = ner_tokenizer.encode(text, truncation=True, max_length=max_len, return_tensors="pt")
    decoded = ner_tokenizer.decode(encoded[0], skip_special_tokens=True)
    return decoded

def extract_entities(text):
    try:
        truncated_text = truncate_text_for_ner(text)
        entities = ner(truncated_text)
        locations = [e["word"] for e in entities if e["entity_group"] == "LOC"]
        cleaned_locations = [re.sub(r'##', '', loc) for loc in locations]
        return ", ".join(cleaned_locations) if cleaned_locations else "N/A"
    except Exception as e:
        print(f"Error extracting entities from text: {text[:50]}... | Error: {e}")
        return "N/A"

data["locations"] = data["text"].apply(extract_entities)
print("Posts with Extracted Locations:")
print(data[["platform", "text", "locations"]])

# Save the analyzed data
data.to_csv("reddit_osint_analyzed.csv", index=False)
print("Analyzed data saved to reddit_osint_analyzed.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'reddit_osint_data.csv'